# Validation of public tranport point

In [1]:
import json
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import pandas as pd
from shapely.geometry import Polygon, LineString
import osmnx as ox
import numpy as np

from destination_validation import *


In [2]:
# configure script
cities = ['seattle']
dest_buffer_dists = [10, 50]
indicators_filepath = 'indicators.csv'
figure_filepath_city = 'fig/city_destination-comparison-{city}.png'
figure_filepath_core = 'fig/core_destination-comparison-{city}.png'

if not os.path.exists('fig/'):
    os.makedirs('fig/')


In [4]:
# RUN THE SCRIPT
indicators = {}
for city in cities:
    print(ox.ts(), f'begin processing {city}')
    indicators[city] = {}
    
    # load this city's configs
    with open(f'configuration/{city}.json') as f:
        config = json.load(f)
        # load destination gdfs from osm graph and official shapefile
        study_area, geopackage, gdf_osm_destinations_clipped, gdf_official_destinations_clipped = load_data(config['osm_buffer_gpkg_path'],
                                                                                                            config['osm_dest_name'],
                                                                                                            config['official_dests_filepath'])
        # plot map of study area + osm and official destinations, save to disk
        fp_city = figure_filepath_city.format(city=city)
        fig, ax = plot_city_data(gdf_osm_destinations_clipped, gdf_official_destinations_clipped, study_area, fp_city)

        # calculate the convex hull to get city core
        osm_core_dests = get_core_dests(geopackage, 0.1, study_area, gdf_osm_destinations_clipped)
        official_core_dests = get_core_dests(geopackage, 0.1, study_area, gdf_official_destinations_clipped)
        indicators[city]['osm_core_dests_count'] = len(osm_core_dests)
        indicators[city]['official_core_dests_count'] = len(official_core_dests)
        print(ox.ts(), 'created core for osm/official destinations')

        # plot map of study area and core destinations
        fp_core = figure_filepath_core.format(city=city)
        fig, ax = plot_core_data(osm_core_dests, official_core_dests, gdf_osm_destinations_clipped, study_area, fp_core)

        # calculate total destination count in each dataset, then add to indicators
        osm_dest_count = len(gdf_osm_destinations_clipped)
        official_dest_count = len(gdf_official_destinations_clipped)
        indicators[city]['osm_dest_count'] = osm_dest_count
        indicators[city]['official_dest_count'] = official_dest_count
        print(ox.ts(), 'calculated destination counts')

        # calculate the % overlaps of areas and lengths between osm and official destinations with different buffer distances
        for dist in dest_buffer_dists:
            osm_buff_prop, official_buff_prop = calculate_intersect(gdf_osm_destinations_clipped, gdf_official_destinations_clipped, dist)
            indicators[city][f'osm_buff_overlap_count_{dist}'] = osm_buff_prop
            indicators[city][f'official_buff_overlap_count_{dist}'] = official_buff_prop
            print(ox.ts(), f'calculated destination overlaps for buffer {dist}')

# turn indicators into a dataframe and save to disk
df_ind = pd.DataFrame(indicators).T
df_ind.to_csv(indicators_filepath, index=True, encoding='utf-8')
print(ox.ts(), f'all done, saved indicators to disk at "{indicators_filepath}"')

2020-07-23 18:01:35 begin processing seattle
2020-07-23 18:01:35 loaded study area boundary
2020-07-23 18:01:39 loaded and filtered official destinations shapefile
2020-07-23 18:01:39 loaded osm destinations shapefile
2020-07-23 18:01:42 clipped osm/official destinations to study area boundary
2020-07-23 18:01:44 figure saved to disk at "fig/city_destination-comparison-seattle.png"
2020-07-23 18:01:48 created core for osm/official destinations
2020-07-23 18:01:48 figure saved to disk at "fig/core_destination-comparison-seattle.png"
2020-07-23 18:01:48 calculated destination counts
2020-07-23 18:10:45 calculated destination overlaps for buffer 10
2020-07-23 18:19:43 calculated destination overlaps for buffer 50
2020-07-23 18:19:43 all done, saved indicators to disk at "indicators.csv"
